In [30]:
from mmdew.bucket_stream2 import BucketStream
from mmdew.bucket_stream_old import BucketStream as OldBucketStream
import numpy as np
import pandas as pd
from mmdew.mmd import MMD
mymmd = MMD(biased=True, gamma=1)
from sklearn import metrics

X = np.random.normal(0, 1, 32)
Y = np.random.normal(1, 2, 32)


m_idx = np.random.default_rng().integers(32, size=5)
subsample = X[m_idx]
start_weights = (1/32) * metrics.pairwise.linear_kernel(subsample.reshape(-1,1),subsample.reshape(-1,1)) @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1),X.reshape(-1,1)) @ np.ones(32)

print(f"real mmd:{mymmd.mmd(X.reshape(-1,1),Y.reshape(-1,1))}")

end_weights = np.ones(32)/32
addend_1 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), subsample.reshape(-1,1)) @ start_weights
addend_2 = end_weights.T @ metrics.pairwise.linear_kernel(Y.reshape(-1,1), Y.reshape(-1,1)) @ end_weights
addend_3 = start_weights.T @ metrics.pairwise.linear_kernel(subsample.reshape(-1,1), Y.reshape(-1,1)) @ end_weights

print(f" nyström mmd: {(addend_1 + addend_2 - (2 * addend_3))}")


real mmd:0.7635437624508594
 nyström mmd: 0.7024237833737192


In [12]:
i = 0
bs_ss.buckets[i].elements


array([[[1]]])

In [13]:
bs_no_ss.buckets[i].capacity


1

In [14]:
bs_ss.buckets[i].uncompressed_capacity

1